In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os.path import join
import os
print(os.getcwd())
from sklearn.model_selection import train_test_split
from tensorflow import keras
import tensorflow as tf
# Path for my Archive folder
from config import Archive_Path


physical_devices = tf.config.list_physical_devices() 
for x in physical_devices:
    try:
        tf.config.experimental.set_memory_growth(x, True)
    except:
        print(x, "\n")

c:\Users\migue\Desktop\estagio_lip_temp\ML
Num GPUs Available:  0
PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU') 

PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU') 

PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU') 



### Importar os Dados

In [2]:
data_background = pd.read_hdf(join(Archive_Path,"Data.h5"), key='bkgd').reset_index(drop=True)
data_signal = pd.read_hdf(join(Archive_Path,"Data.h5"), key='signal').reset_index(drop=True)

KeyboardInterrupt: 

In [ ]:
data_signal.describe()

In [ ]:
data_background.describe()

### Os dados ainda não estao prontos, basta olhar para a mean != 0..

In [8]:
# Adicionar uma coluna de Signal -> 0 se for background e 1 se for signal
data_background['Signal'] = pd.Series([0 for x in range(len(data_background.index))])
data_signal['Signal'] = pd.Series([1 for x in range(len(data_signal.index))])

In [9]:
# Vamos dar juntar tudo na variavel "Data" - Tem os dados, as labels e os Weights
Data = data_background.append(data_signal).rename(columns={'gen_weights':'Weights'})

In [10]:
Data.describe()

,Electron1_Eta,Electron1_PT,Electron1_Phi,Electron2_Eta,Electron2_PT,Electron2_Phi,Electron_Multi,FatJet1_Eta,FatJet1_Mass,FatJet1_PT,...,Muon1_Eta,Muon1_PT,Muon1_Phi,Muon2_Eta,Muon2_PT,Muon2_Phi,Muon_Multi,ScalarHT_HT,Weights,Signal
count,4.440800e+06,4.440800e+06,4.440800e+06,4.440800e+06,4.440800e+06,4.440800e+06,4.440800e+06,4.440800e+06,4.440800e+06,4.440800e+06,...,4.440800e+06,4.440800e+06,4.440800e+06,4.440800e+06,4.440800e+06,4.440800e+06,4.440800e+06,4.440800e+06,4.440800e+06,4.440800e+06
mean,4.642362e-06,4.102770e+01,1.146982e-03,3.100396e-06,1.036529e+01,-4.581339e-06,8.324586e-01,3.533756e-04,8.693247e+00,3.873254e+01,...,3.026105e-04,5.423478e+01,-1.956640e-04,-5.599270e-05,1.706459e+01,-1.866145e-05,1.196289e+00,3.714505e+02,7.766081e-06,4.407427e-02
std,8.472479e-01,5.473847e+01,1.371933e+00,5.798893e-01,2.252014e+01,9.181378e-01,8.186355e-01,4.162720e-01,2.652501e+01,1.099466e+02,...,9.976884e-01,5.717222e+01,1.571230e+00,7.815677e-01,2.605593e+01,1.198585e+00,8.223965e-01,2.021479e+02,1.398379e-05,2.052602e-01
min,-2.499997e+00,0.000000e+00,-3.141592e+00,-2.499985e+00,0.000000e+00,-3.141592e+00,0.000000e+00,-4.000467e+00,-6.103516e-05,0.000000e+00,...,-2.399990e+00,0.000000e+00,-3.141592e+00,-2.399981e+00,0.000000e+00,-3.141588e+00,0.000000e+00,4.447565e+01,1.286454e-09,0.000000e+00
25%,-1.986022e-01,0.000000e+00,-3.912720e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,-5.680689e-01,1.020456e+01,-1.047653e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,2.342856e+02,1.811693e-07,0.000000e+00
50%,0.000000e+00,2.898474e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,4.799751e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,3.308271e+02,5.028781e-06,0.000000e+00
75%,2.005566e-01,6.450357e+01,3.971428e-01,0.000000e+00,1.263088e+01,0.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,5.684932e-01,7.523514e+01,1.046757e+00,0.000000e+00,3.088431e+01,0.000000e+00,2.000000e+00,4.581491e+02,7.089469e-06,0.000000e+00
max,2.499979e+00,2.124003e+03,3.141593e+00,2.499981e+00,9.984486e+02,3.141575e+00,5.000000e+00,3.873402e+00,7.324354e+02,2.939618e+03,...,2.399990e+00,2.646508e+03,3.141587e+00,2.399961e+00,1.217265e+03,3.141591e+00,5.000000e+00,4.926052e+03,8.083906e-05,1.000000e+00


In [11]:
# Apagar variaveis para poupar espaço
del data_background
del data_signal

## Data preprocessing done! Let's start with the ML model

In [12]:
# Separar em training e test set
X_train, X_test, y_train, y_test = train_test_split(
        Data.drop(columns=['Signal']), 
        Data['Signal'], 
        test_size=0.5)
# Release RAM
del Data

Standardization of the data

In [13]:
class Standardization(keras.layers.Layer):
    def adapt(self, data_sample):
        self.means_ = np.mean(data_sample, axis=0, keepdims=True)
        self.stds_ = np.std(data_sample, axis=0, keepdims=True)
    def call(self, inputs):
        return (inputs - self.means_) / (self.stds_ + keras.backend.epsilon())
        
std_layer = Standardization()
std_layer.adapt(X_train.drop(columns=['Weights']).to_numpy())

Definir as class_weights para os dados de treino

In [14]:
class_weights = {
    0: 1,
    1: len([x for x in y_train if x == 0]) / len([x for x in y_train if x == 1]) # Nº de elementos da classe de referencia a dividir pelo nº de elementos da classe atual
}

In [15]:
class_weights

{0: 1, 1: 21.65459999387824}

Build the model

In [16]:
model = keras.models.Sequential(
    [
     keras.layers.Input(shape=(69,)),
     std_layer,
     keras.layers.Dense(100, activation='relu'),
     keras.layers.Dense(100, activation='relu'),
     keras.layers.Dense(100, activation='relu'),
     keras.layers.Dense(1, activation='sigmoid')
    ]
  )

model.compile(
              optimizer='adam', 
              loss="binary_crossentropy", 
              metrics=["accuracy", keras.metrics.AUC()]
              )

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
standardization (Standardiza (None, 69)                0         
_________________________________________________________________
dense (Dense)                (None, 100)               7000      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 27,301
Trainable params: 27,301
Non-trainable params: 0
_________________________________________________________________


In [18]:
import time
# Copiado do Gilberto
# Model name
name = str(time.time())

# Tensorboard
TB = keras.callbacks.TensorBoard("logs/" + name, write_images=True)

# Early Stopping
ES = keras.callbacks.EarlyStopping(monitor="loss", patience=6, verbose=2, mode="min")

# Model Checkpoint
MC = keras.callbacks.ModelCheckpoint("models/" + name + ".h5", save_best_only=True, monitor="val_loss",
                                     mode="min")

In [19]:

history = model.fit(
    x=X_train.drop(columns=['Weights']).to_numpy(),
    y=y_train,
    epochs=10,
    #batch_size=1024,
    shuffle=True,
    class_weight=class_weights,
    callbacks=[TB, ES, MC],
    sample_weight = X_train['Weights'].to_numpy()
    )
        

SyntaxError: invalid syntax (<ipython-input-19-758fff2ebfc6>, line 7)

### Saving the Model

In [20]:
model.save_weights("weights")
model.save("model")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: model\assets


In [21]:
model.load_weights("weights")

## Metrics

In [22]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score

In [23]:
y_preds = model.predict(X_test.drop(columns=['Weights']).values, batch_size=1024).reshape(-1)

In [24]:
accuracy_score(y_test.values, y_preds.round())

0.05356332192397766

In [25]:
confusion_matrix(y_test.values, y_preds.round(), normalize="all")

array([[9.91307872e-03, 9.46079535e-01],
       [3.57142857e-04, 4.36502432e-02]])

In [26]:
confusion_matrix(y_test.values, y_preds.round())

array([[  22011, 2100675],
       [    793,   96921]], dtype=int64)

In [27]:
precision_score(y_test.values, y_preds.round())

0.04410319276154489

In [34]:
recall_score(y_test.values, y_preds.round())

NameError: name 'recall_score' is not defined

## Plotting Roc Curve

In [35]:
from sklearn.metrics import roc_curve

fpr, tpr, thr = roc_curve(y_test, y_preds)

plt.plot(fpr, tpr, label='ROC')
plt.plot([0,1],[0,1], 'k--')
plt.show()

ModuleNotFoundError: No module named 'sklearn'